In [6]:
'''
cat data_aux/cfgs_run | xargs -I @ -P 10 python3 -u run2out.py -c @ > log/run2out.out & 
'''
import os, click, h5py, re, pickle
import numpy as np

runPath='/capstor/store/cscs/userlab/s1174/lyan/code/scratch/run/03_NpiScatteringWilson/A15/'

lat_L,lat_T=48,48

moms_target=[[0,0,0, 0,0,0, 0,0,0, 0,0,0]] # pi2,pf1,pf2,pc=000

type2base={
    '11':'gc,gd,abdc->ab',
    '12':'dc,gd,bagc->ab',
    '13':'bc,gd,dagc->ab',
    '21':'bc,gd,agdc->ab',
    '22':'dc,gd,gabc->ab',
    '23':'gc,gd,dabc->ab',
}
def type2pat_V3V24(tp,extra_V3,extra_V24,extra_V3V24):
    t=type2base[tp]
    t1,t2=t.split('->')
    t11,t12,t13=t1.split(',')
    return f'{t11}{extra_V3},{t12},{t13}{extra_V24}->{t2}{extra_V3V24}' 

def key2mom(mom_key):
    t1,t2=mom_key.split('=')
    x,y,z=t2.split('_')
    x,y,z=int(x),int(y),int(z)
    return [x,y,z]
def moms2dic(moms):
    dic={}
    for i,mom in enumerate(moms):
        dic[tuple(mom)]=i
    return dic

Format_ab=4
a2ab_map=[a for a in range(Format_ab) for b in range(Format_ab)]
b2ab_map=[b for a in range(Format_ab) for b in range(Format_ab)]

gamma_1=gamma_x=np.array([[0.,0.,0.,1j],[0.,0.,1j,0.],[0.,-1j,0.,0.],[-1j,0.,0.,0.]])
gamma_2=gamma_y=np.array([[0.,0.,0.,1.],[0.,0.,-1.,0.],[0.,-1.,0.,0.],[1.,0.,0.,0.]])
gamma_3=gamma_z=np.array([[0.,0.,1j,0.],[0.,0.,0.,-1j],[-1j,0.,0.,0.],[0.,1j,0.,0.]])
gamma_4=gamma_t=np.array([[1.,0.,0.,0.],[0.,1.,0.,0.],[0.,0.,-1.,0.],[0.,0.,0.,-1.]])
gamma_5=(gamma_1@gamma_2@gamma_3@gamma_4)

cg5=1j*gamma_2@gamma_4@gamma_5
Gm_i1=cg5

def get_phase(src_int,mom):
    (sx,sy,sz,st)=src_int
    return np.exp(1j*(2*np.pi/lat_L)*(np.array([sx,sy,sz])@mom))
def mom2pi1(mom):
    pi2=mom[0:3]; pf1=mom[3:6]; pf2=mom[6:9]; pc=mom[9:12]
    pi1=np.array(pf1)+pf2+pc-pi2
    return pi1

def run(cfg):
    inpath=f'{runPath}run/{cfg}/'
    outpath=f'{runPath}out/{cfg}/'
    os.makedirs(outpath,exist_ok=True)
    
    files=[file for file in os.listdir(inpath) if file.endswith('.h5')]
    
    for file in files:
        if file.endswith('NpiScatteringWilson.h5'):
            infile=f'{inpath}{file}'
            outfile=f'{outpath}{file}'
            outfile_flag=outfile+'_flag'
            if (not os.path.isfile(outfile)) or os.path.isfile(outfile_flag):
                with open(outfile_flag,'w') as f:
                    pass
                with h5py.File(outfile,'w') as fw, h5py.File(infile) as f:
                    src=list(f.keys())[0]
                    (sx,sy,sz,st)=re.search('sx([0-9]*)sy([0-9]*)sz([0-9]*)st([0-9]*)',src).groups()
                    (sx,sy,sz,st)=(int(sx),int(sy),int(sz),int(st))
                    src_new='sx{:03d}sy{:03d}sz{:03d}st{:03d}'.format(sx,sy,sz,st)
                    Nstoc=len([stoc for stoc in f[f'{src}/V2B_1'].keys() if stoc.startswith('i_stoc=')])
                    
                    pi2s_key=[pi2 for pi2 in f[f'{src}/V3B_1'].keys() if pi2.startswith('pi2=')]; pi2s_key.sort()
                    pi2s=[key2mom(pi2) for pi2 in pi2s_key]
                    pf1s=[list(mom) for mom in np.atleast_2d(f[f'{src}/V2B_1/mvec'])]
                    pf2s=[list(mom) for mom in np.atleast_2d(f[f'{src}/V3B_1/mvec'])]
                    
                    pi2Map=[pi2s.index(mom[:3]) for mom in moms_target]
                    pf1Map=[pf1s.index(mom[3:6]) for mom in moms_target]
                    pf2Map=[pf2s.index(mom[6:9]) for mom in moms_target]
                    
                    phases=np.array([get_phase((sx,sy,sz,st),mom2pi1(mom)) for mom in moms_target])
                    
                    # print(f[f'{src}/V2B1/i_stoc=0'])
                    # print(f[f'{src}/V3B1/pi2=0_0_0/i_stoc=0'])
                    # print(f[f'{src}/V4B1/i_stoc=0'])
                    
                    # print(f[f'{src}/V2W1a/pi2=0_0_0/i_stoc=0'])
                    # print(f[f'{src}/V3W1/i_stoc=0'])
                    # print(f[f'{src}/V4W1a/pi2=0_0_0/i_stoc=0'])
                    
                    notes=['time,mom,dirac,stoc','mom=[pi2,pf1,pf2]','dirac=4a+b','stoc=i or i*Nstoc+j (Nstoc=sqrt(len(stoc)))']
                    fw.create_dataset('notes',data=notes)
                    fw.create_dataset('moms',data=moms_target)
                    
                    conts=['B114_1','B122_1','B132_1','B214_1','B232_1',
                           'B114_2','B122_2','B132_2','B214_2','B232_2',
                           'Z114_1','Z122_1','Z132_1','Z214_1','Z222_1']
                    for cont in conts:
                        topo=cont[0]; tp=cont[1:3]; V24=cont[3]; md=cont[-1]
                        kyV3=f'V3{topo}_{md}'; kyV24=f'V{V24}{topo}_{md}'; kyV3V24=f'{cont}'
                        if topo=='W':
                            seqWhere=cont[4]
                            kyV24=f'V{V24}{topo}{seqWhere}_{md}'
                        t_V3=np.array([[f[f'{src}/{kyV3}/{pi2_key}/i_stoc={istoc}'][:,:,0] for pi2_key in pi2s_key] for istoc in range(Nstoc)])
                        t_V3=t_V3[...,0]+1j*t_V3[...,1]
                        t_V3=np.transpose(t_V3,[4,5,2,3,1,0]) # spin,color, time,xmom=pf2,pi2,stoc
                        t_V24=np.array([f[f'{src}/{kyV24}/i_stoc={istoc}'][:,:,0] for istoc in range(Nstoc)])
                        t_V24=t_V24[...,0]+1j*t_V24[...,1]
                        t_V24=np.transpose(t_V24,[3,4,5,6,1,2,0]) # spin1,spin2,spin3,color, time,ymom=pf1,stoc
                        pat=type2pat_V3V24(tp,'txps','tys','tpyxs')
                        t=np.einsum(pat,t_V3,Gm_i1,t_V24,optimize='optimal') # alpha,beta,time,pi2,pf1,pf2,stoc
                        t=np.transpose(t,[2,3,4,5,0,1,6]) # time,pi2,pf1,pf2,alpha,beta,stoc
                        t=t[:,pi2Map,pf1Map,pf2Map]
                        t=t*phases[None,:]
                        t=t[:,:,a2ab_map,b2ab_map]
                        fw.create_dataset(f'{src_new}/{kyV3V24}',data=t.astype('complex64'))
                        
                    conts=['W1141_1','W1142_1','W1221_1','W1222_1','W1321_1','W1322_1','W2321_1',
                           'W1141_2','W1142_2','W1221_2','W1222_2','W1321_2','W1322_2','W2321_2']
                    for cont in conts:
                        topo=cont[0]; tp=cont[1:3]; V24=cont[3]; md=cont[-1]
                        kyV3=f'V3{topo}_{md}'; kyV24=f'V{V24}{topo}_{md}'; kyV3V24=f'{cont}'
                        if topo=='W':
                            seqWhere=cont[4]
                            kyV24=f'V{V24}{topo}{seqWhere}_{md}'
                        
                        t_V3=np.array([f[f'{src}/{kyV3}/i_stoc={istoc}'][:,:,0] for istoc in range(Nstoc)])
                        t_V3=t_V3[...,0]+1j*t_V3[...,1]
                        t_V3=np.transpose(t_V3,[3,4,1,2,0]) # spin,color, time,xmom=pf2,stoc
                        t_V24=np.array([[f[f'{src}/{kyV24}/{pi2_key}/i_stoc={istoc}'][:,:,0] for pi2_key in pi2s_key] for istoc in range(Nstoc)])
                        t_V24=t_V24[...,0]+1j*t_V24[...,1]
                        t_V24=np.transpose(t_V24,[4,5,6,7,2,3,1,0]) # spin1,spin2,spin3,color, time,ymom=pf1,pi2,stoc
                        pat=type2pat_V3V24(tp,'txs','typs','tpyxs')
                        t=np.einsum(pat,t_V3,Gm_i1,t_V24,optimize='optimal') # alpha,beta,time,pi2,pf1,pf2,stoc
                        t=np.transpose(t,[2,3,4,5,0,1,6]) # time,pi2,pf1,pf2,alpha,beta,stoc
                        t=t[:,pi2Map,pf1Map,pf2Map]
                        t=t*phases[None,:]
                        t=t[:,:,a2ab_map,b2ab_map]
                        fw.create_dataset(f'{src_new}/{kyV3V24}',data=t.astype('complex64'))

        # os.remove(outfile_flag)
        
    print('flag_cfg_done: '+cfg)
    
run('a0050')

flag_cfg_done: a0050
